In [1]:
from IPython.display import clear_output

!python -m spacy download it_core_news_sm
!pip install bs4
!pip install rdflib
!pip install SPARQLWrapper

clear_output()
print("##########")

##########


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
%cd /content/drive/MyDrive/TSsrc/

/content/drive/MyDrive/TSsrc


In [7]:
import spacy, it_core_news_sm
from spacy import displacy
from pathlib import Path
from SPARQLWrapper import SPARQLWrapper, JSON, N3

def cleanUp(word):
  newWord = word.replace(" ","_")
  newWord = newWord.replace("'", "")
  newWord = newWord.replace(".", "")
  newWord = newWord.replace("#", "")
  return newWord

def writeRDFa(txt, res, resNew, wordNER):
  if (res == resNew):
    res = ""
    if(wordNER.label_ == "PER"):
      txt = txt.replace(wordNER.text, '<span vocab="http://xmlns.com/foaf/0.1/" typeof="Person"><span property="name">' + wordNER.text + '</span></span>')
    elif(wordNER.label_ == "LOC"):
      txt = txt.replace(wordNER.text, '<span vocab="http://schema.org/" typeof="Place"><span property="name">' + wordNER.text + '</span></span>')
    elif(wordNER.label_ == "ORG"):
      txt = txt.replace(wordNER.text, '<span vocab="http://schema.org/" typeof="SportsOrganization"><span property="name">' + wordNER.text + '</span></span>')
  else:
     if(wordNER.label_ == "PER"):
       txt = txt.replace(wordNER.text, '<span vocab="http://xmlns.com/foaf/0.1/" prefix="rdfs: http://www.w3.org/2000/01/rdf-schema" typeof="Person"><span property="name"><span property="rdfs:seeAlso" = "' + res + '"/>' + wordNER.text + '</span></span>')
     elif(wordNER.label_ == "LOC"):
       txt = txt.replace(wordNER.text, '<span vocab="http://schema.org/" prefix="rdfs: http://www.w3.org/2000/01/rdf-schema" typeof="Place"><span property="name"><span property="rdfs:seeAlso" = "' + res + '"/>' + wordNER.text + '</span></span>')
     elif(wordNER.label_ == "ORG"):
       txt = txt.replace(wordNER.text, '<span vocab="http://schema.org/" prefix="rdfs: http://www.w3.org/2000/01/rdf-schema" typeof="SportsOrganization"><span property="name"><span property="rdfs:seeAlso" = "' + res + '"/>' + wordNER.text + '</span></span>')
  return txt

nlp = it_core_news_sm.load()
sparql = SPARQLWrapper('https://dbpedia.org/sparql')
txt = Path('stadio.html').read_text()

print("##########")
NER_first = nlp(txt)
processed = ['A']

for wordNER in NER_first.ents:
  if wordNER.text not in processed:
    if(wordNER.label_ == "PER" or wordNER.label_ == "ORG" or wordNER.label_ == "LOC"):
      newWord = cleanUp(wordNER.text)
      sparql.setQuery(f'''
      SELECT ?object 
        WHERE {{ dbr:{newWord} foaf:isPrimaryTopicOf ?object.
      }}''')
      sparql.setReturnFormat(JSON)
      qres = sparql.query().convert()
      res = ""
      resNew = res
      for result in qres['results']['bindings']:
        value = result['object']['value']
        res = (result['object']['value'])
      txt = writeRDFa(txt, res, resNew, wordNER)
  processed.append(wordNER.text)

f = open("stadioOutput1.html", "w")
f.write(txt)
f.close()

#displacy.render(NER_first,style="ent",jupyter=True)
print("%%%%%%%%%%")


##########
%%%%%%%%%%
